In [2]:
from openai import OpenAI
import ollama
import gradio as gr
import json

def get_response(messages):
    openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
    response = openai.chat.completions.create(model="gemma2:2b",messages=messages, stream=False)
    return response.choices[0].message.content


In [3]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")
    

In [5]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}
tools = [{"type": "function", "function": price_function}]

In [14]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    # does not work with open source models yet
    # openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
    # response = openai.chat.completions.create(model="gemma2:2b",messages=messages, stream=False, tools=tools)

    import os
    os.environ['OPENAI_API_KEY'] = "provide_key_here"
    MODEL = "gpt-4o-mini"
    openai = OpenAI()
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model="gemma2:2b", messages=messages)
    
    return response.choices[0].message.content

In [15]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": message.tool_calls[0].id
    }
    return response, city

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

In [17]:
print("""
User: Hey there, I'd like to go to London
Agent: Would you like to book a trip to London
User: Yes
Agent: Trip to london would be approx 799
..
User: How about Timbaktoo
Agent: Sorry, I don't have this information
""")


User: Hey there, I'd like to go to London
Agent: Would you like to book a trip to London
User: Yes
Agent: Trip to london would be approx 799
..
User: How about Timbaktoo
Agent: Sorry, I don't have this information

